In [29]:
import pandas as pd
import re
import tldextract
from scipy.sparse import coo_matrix, hstack

In [30]:
df = pd.read_csv('url _classification.csv', header = None)
df.columns = ["index" , "url" , "label"]
# df["url"]= df["URL"].astype(str)
# data = pd.DataFrame(df)
# data.dtypes
df = df.dropna()
df = df.reset_index(drop=True)
print(df.isnull().values.any())

False


In [31]:
def split_protocol(url:str):
    delim = "://"
    if delim in url:
        n = url.index(delim)
        return [url[:n], url[n + len(delim):]]
    else:
        return ['', url]
    return ['', url]

split_protocol_data = df['url'].apply(lambda x: split_protocol(x))
df = pd.concat([df, pd.DataFrame(split_protocol_data.tolist(), columns=['protocol', 'url_2'])], axis=1)
# split_protocol_data = dt['url'].apply(lambda x: split_protocol(x))
# dt = pd.concat([dt, pd.DataFrame(split_protocol_data.tolist(), columns=['protocol', 'url_2'])], axis=1)

In [32]:
def generalize_number(input: str): 
    n = len(input)
    output = ""
    prev_is_number = False
    nums = []
    for i in range(10):
        nums.append(str(i))
    for i in range(n):
        if input[i] in nums:
            if prev_is_number:
                continue
            else:
                output = output + '1'
                prev_is_number = True
        else:
            output = output + input[i]
            prev_is_number = False
    return output

no__num_data = list(map(lambda url: generalize_number(url),  df['url_2']))
df = pd.concat([df, pd.DataFrame(no__num_data, columns=['url_no_num'])], axis=1)


# no__num_data = list(map(lambda url: generalize_number(url),  dt['url_2']))
# dt = pd.concat([dt, pd.DataFrame(no__num_data, columns=['url_no_num'])], axis=1)

In [33]:
df[df['url_2'] != df['url_no_num']]
# dt[dt['url_2'] != dt['url_no_num']]

,index,url,label,protocol,url_2,url_no_num
6,7,http://www2.117.ne.jp/~mb1996ax/enadc.html,Adult,http,www2.117.ne.jp/~mb1996ax/enadc.html,www1.1.ne.jp/~mb1ax/enadc.html
14,15,http://www9.kinghost.com/fetish/hentaibee/,Adult,http,www9.kinghost.com/fetish/hentaibee/,www1.kinghost.com/fetish/hentaibee/
16,17,http://www.geocities.com/kaseychan17/index.html,Adult,http,www.geocities.com/kaseychan17/index.html,www.geocities.com/kaseychan1/index.html
18,19,http://e.webring.com/hub?sid=&amp;ring=hentff9...,Adult,http,e.webring.com/hub?sid=&amp;ring=hentff98&amp;i...,e.webring.com/hub?sid=&amp;ring=hentff1&amp;id...
23,24,http://www.geocities.com/eddie_monsoon2002/,Adult,http,www.geocities.com/eddie_monsoon2002/,www.geocities.com/eddie_monsoon1/
...,...,...,...,...,...,...
1562935,1562939,http://ejmas.com/jwma/articles/2000/jwmaart_ro...,Sports,http,ejmas.com/jwma/articles/2000/jwmaart_roberts_0...,ejmas.com/jwma/articles/1/jwmaart_roberts_1.htm
1562936,1562940,http://ejmas.com/jwma/articles/2000/jwmaart_pf...,Sports,http,ejmas.com/jwma/articles/2000/jwmaart_pfrenger_...,ejmas.com/jwma/articles/1/jwmaart_pfrenger_1.htm
1562937,1562941,http://ejmas.com/jwma/articles/2000/jwmaart_ka...,Sports,http,ejmas.com/jwma/articles/2000/jwmaart_kautz_010...,ejmas.com/jwma/articles/1/jwmaart_kautz_1.htm
1562939,1562943,http://ejmas.com/jalt/jaltart_baxter_0102.htm,Sports,http,ejmas.com/jalt/jaltart_baxter_0102.htm,ejmas.com/jalt/jaltart_baxter_1.htm


In [34]:
def split_prefix(url:str):
    pattern = re.compile('^(?P<prefix>w*[\d]*)\.(?P<fulldomain>.*)')
    match = re.match(pattern, url)
    if match is None:
        return ['', url]
    else:
        split = match.groupdict()
        return [split['prefix'], split['fulldomain']]

split_prefix_data = list(map(lambda url: split_prefix(url),  df['url_no_num']))
df = pd.concat([df, pd.DataFrame(split_prefix_data, columns=['prefix', 'fulldomain'])], axis=1)


# split_prefix_data = list(map(lambda url: split_prefix(url),  dt['url_no_num']))
# dt = pd.concat([dt, pd.DataFrame(split_prefix_data, columns=['prefix', 'fulldomain'])], axis=1)

In [35]:
df = df.drop(columns=['url_2'])
# dt = dt.drop(columns=['url_2'])

In [36]:
def split_url_uri(url:str):
    delim = "/"
    if delim in url:
        n = url.index(delim)
        return [url[:n], url[n + len(delim):]]
    else:
        return [url, '']

split_url_uri_data = df['fulldomain'].apply(lambda x: split_url_uri(x))
df = pd.concat([df, pd.DataFrame(split_url_uri_data.tolist(), columns=['urlonly', 'directory'])], axis=1)

# split_url_uri_data = dt['fulldomain'].apply(lambda x: split_url_uri(x))
# dt = pd.concat([dt, pd.DataFrame(split_url_uri_data.tolist(), columns=['urlonly', 'uri'])], axis=1)

In [37]:
def split_url(url):
    # extract subdomain, domain, and domain suffix from url
    # if item == '', fill with '<empty>'
    subdomain, domain, domain_suffix = ('' if extracted == '' else extracted for extracted in tldextract.extract(url))
    
    return [subdomain, domain, domain_suffix]

# parsed url
extract_url_data = [split_url(url) for url in df['fulldomain']]
extract_url_data = pd.DataFrame(extract_url_data, columns=['subdomain', 'domain', 'domain_suffix'])


# extract_url_data = [split_url(url) for url in dt['fulldomain']]
# extract_url_data = pd.DataFrame(extract_url_data, columns=['subdomain', 'domain', 'domain_suffix'])

# concat extracted feature with main data
data = df.reset_index(drop=True)
df = pd.concat([df, extract_url_data], axis=1)

# data = dt.reset_index(drop=True)
# dt = pd.concat([dt, extract_url_data], axis=1)

In [38]:
df['domain_suffix'] = df['domain_suffix'].apply(lambda x: x.replace('.', ' '))
df['domain_suffix'] = df['domain_suffix'].apply(lambda x: x.strip())

df['directory'] = df['directory'].apply(lambda x: x.replace('.', ' '))
df['directory'] = df['directory'].apply(lambda x: x.replace('/', ' '))
df['directory'] = df['directory'].apply(lambda x: x.replace('?', ' '))
df['directory'] = df['directory'].apply(lambda x: x.replace('=', ' '))
df['directory'] = df['directory'].apply(lambda x: x.replace('-', ' '))
df['directory'] = df['directory'].apply(lambda x: x.strip())




# dt['domain_suffix'] = dt['domain_suffix'].apply(lambda x: x.replace('.', ' '))
# dt['domain_suffix'] = dt['domain_suffix'].apply(lambda x: x.strip())

# dt['uri'] = dt['uri'].apply(lambda x: x.replace('.', ' '))
# dt['uri'] = dt['uri'].apply(lambda x: x.replace('/', ' '))
# dt['uri'] = dt['uri'].apply(lambda x: x.replace('?', ' '))
# dt['uri'] = dt['uri'].apply(lambda x: x.replace('=', ' '))
# dt['uri'] = dt['uri'].apply(lambda x: x.replace('-', ' '))
# dt['uri'] = dt['uri'].apply(lambda x: x.strip())

In [39]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import f1_score, classification_report

In [40]:
unique_labels = set(df['label'])
label2idx = {k: i for i, k in enumerate(unique_labels)}
idx2label = {i: k for k, i in label2idx.items()}

# Multiclass classification
df['label'] = df['label'].map(label2idx)

# Binary Classification
# df['label'] = df['label'].apply(lambda x: 0 if x == 'Adult' else 1)


# unique_labels = set(dt['label'])
# label2idx = {k: i for i, k in enumerate(unique_labels)}
# idx2label = {i: k for k, i in label2idx.items()}

# # Multiclass classification
# dt['label'] = dt['label'].map(label2idx)
df.tail(10)

,index,url,label,protocol,url_no_num,prefix,fulldomain,urlonly,directory,subdomain,domain,domain_suffix
1562965,1562969,http://www.aodonline.org/chsl/chsl.htm,13,http,www.aodonline.org/chsl/chsl.htm,www,aodonline.org/chsl/chsl.htm,aodonline.org,chsl chsl htm,,aodonline,org
1562966,1562970,http://aenwebsites.com/tsl/,13,http,aenwebsites.com/tsl/,,aenwebsites.com/tsl/,aenwebsites.com,tsl,,aenwebsites,com
1562967,1562971,http://www.fciac.net/,13,http,www.fciac.net/,www,fciac.net/,fciac.net,,,fciac,net
1562968,1562972,http://www.infosports.com/,13,http,www.infosports.com/,www,infosports.com/,infosports.com,,,infosports,com
1562969,1562973,http://www.hssp.cc/,13,http,www.hssp.cc/,www,hssp.cc/,hssp.cc,,,hssp,cc
1562970,1562974,http://www.maxpreps.com/,13,http,www.maxpreps.com/,www,maxpreps.com/,maxpreps.com,,,maxpreps,com
1562971,1562975,http://www.myscore.com/,13,http,www.myscore.com/,www,myscore.com/,myscore.com,,,myscore,com
1562972,1562976,http://sportsillustrated.cnn.com/highschool,13,http,sportsillustrated.cnn.com/highschool,,sportsillustrated.cnn.com/highschool,sportsillustrated.cnn.com,highschool,sportsillustrated,cnn,com
1562973,1562977,http://rss.cnn.com/rss/si_highschool?format=xml,13,http,rss.cnn.com/rss/si_highschool?format=xml,,rss.cnn.com/rss/si_highschool?format=xml,rss.cnn.com,rss si_highschool format xml,rss,cnn,com
1562974,1562978,http://www.usatoday.com/sports/preps/,13,http,www.usatoday.com/sports/preps/,www,usatoday.com/sports/preps/,usatoday.com,sports preps,,usatoday,com


In [41]:
Training_data = df

In [42]:
training_data, testing_data = train_test_split(Training_data,
                                            test_size=0.2, 
                                            random_state = 2000,
                                            stratify=Training_data['label'])

# GET LABELS
Y_train=training_data['label'].values
Y_test=testing_data['label'].values

In [43]:
training_data['label'].value_counts()

2     203072
12    195154
9     192141
11     94369
10     88229
1      85269
13     81062
5      76216
6      48077
4      46598
7      45181
8      36946
3      28260
14     22615
0       7191
Name: label, dtype: int64

In [44]:
def extract_features(field, df, test_data):
    # TF-IDF BASED FEATURE REPRESENTATION
    tfidf_vectorizer=TfidfVectorizer(use_idf=True, 
                                    analyzer='char_wb',
                                    max_df = 0.95,
                                    ngram_range=(3, 6), 
                                    sublinear_tf=True)
    tfidf_vectorizer.fit_transform(training_data[field].values)

    train_feature_set=tfidf_vectorizer.transform(training_data[field].values)
    test_feature_set=tfidf_vectorizer.transform(testing_data[field].values)
    

    
#     train_feature_set = pad_sequences(train_feature_set, maxlen=10000, padding='post')
#     test_feature_set = pad_sequences(test_feature_set, maxlen=10000, padding='post')


    return train_feature_set,test_feature_set,tfidf_vectorizer
# GET FEATURES
X_train1,X_test1,feature_transformer1=extract_features('domain',training_data,testing_data)
X_train1

<1250380x2963983 sparse matrix of type '<class 'numpy.float64'>'
	with 41393017 stored elements in Compressed Sparse Row format>

In [45]:
def extract_features_subdomain(field, train_data, test_data):
    # TF-IDF BASED FEATURE REPRESENTATION
    tfidf_vectorizer=TfidfVectorizer(use_idf=True, 
                                    analyzer='char_wb',
                                    max_df = 0.95,
                                    ngram_range=(3, 6), 
                                    sublinear_tf=True)
    tfidf_vectorizer.fit_transform(training_data[field].values)

    train_feature_set=tfidf_vectorizer.transform(training_data[field].values)
    test_feature_set=tfidf_vectorizer.transform(testing_data[field].values)
    

    return train_feature_set,test_feature_set,tfidf_vectorizer

X_train2,X_test2,feature_transformer2=extract_features_subdomain('subdomain',training_data,testing_data)
X_train2

<1250380x693364 sparse matrix of type '<class 'numpy.float64'>'
	with 5091754 stored elements in Compressed Sparse Row format>

In [46]:
def extract_features_uri(field, train_data, test_data):
    # TF-IDF BASED FEATURE REPRESENTATION
    tfidf_vectorizer=TfidfVectorizer(use_idf=True, 
                                    analyzer='char_wb',
                                    max_df = 0.95,
                                    ngram_range=(3,6), 
                                    sublinear_tf=True)
    tfidf_vectorizer.fit_transform(training_data[field].values)

    train_feature_set=tfidf_vectorizer.transform(training_data[field].values)
    test_feature_set=tfidf_vectorizer.transform(testing_data[field].values)
    

    return train_feature_set,test_feature_set,tfidf_vectorizer

X_train3,X_test3,feature_transformer3=extract_features_uri('directory',training_data,testing_data)
X_train3

<1250380x2098740 sparse matrix of type '<class 'numpy.float64'>'
	with 28705319 stored elements in Compressed Sparse Row format>

In [47]:
X_train = hstack([X_train1, X_train2, X_train3])
X_test = hstack([X_test1, X_test2, X_test3])

In [48]:
X_test

<312595x5756087 sparse matrix of type '<class 'numpy.float64'>'
	with 17997301 stored elements in Compressed Sparse Row format>

In [50]:
from xgboost import XGBClassifier
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

model = OneVsRestClassifier(XGBClassifier()).fit(X_train, Y_train)
pred = model.predict(X_test)


/Users/sylvesteranthony/miniforge3/envs/tf/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/sylvesteranthony/miniforge3/envs/tf/lib/python3.10/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:12:04] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117899018/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:17:17] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117899018/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:23:03] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117899018/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:29:33] WARNING: /Users/runner/miniforge3/cond

In [51]:
from sklearn.metrics import f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
print(classification_report(Y_test, pred))

              precision    recall  f1-score   support

           0       0.55      0.30      0.39      1798
           1       0.67      0.34      0.46     21317
           2       0.54      0.61      0.57     50768
           3       0.85      0.55      0.67      7065
           4       0.57      0.45      0.50     11649
           5       0.57      0.26      0.36     19054
           6       0.70      0.42      0.52     12020
           7       0.74      0.56      0.64     11296
           8       0.59      0.24      0.34      9236
           9       0.34      0.81      0.48     48035
          10       0.67      0.44      0.53     22057
          11       0.64      0.39      0.48     23592
          12       0.60      0.48      0.53     48788
          13       0.80      0.59      0.68     20266
          14       0.81      0.50      0.62      5654

    accuracy                           0.52    312595
   macro avg       0.64      0.46      0.52    312595
weighted avg       0.59   

In [27]:
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

model = OneVsRestClassifier(LinearSVC()).fit(X_train, Y_train)
pred = model.predict(X_test)

In [28]:
from sklearn.metrics import f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
print(classification_report(Y_test, pred))

              precision    recall  f1-score   support

           0       0.48      0.33      0.39      1798
           1       0.58      0.53      0.55     21317
           2       0.63      0.71      0.66     50768
           3       0.82      0.67      0.73      7065
           4       0.57      0.52      0.54     11649
           5       0.46      0.41      0.43     19054
           6       0.66      0.57      0.61     12020
           7       0.71      0.63      0.67     11296
           8       0.51      0.35      0.42      9236
           9       0.54      0.65      0.59     48035
          10       0.64      0.59      0.62     22057
          11       0.59      0.57      0.58     23592
          12       0.64      0.66      0.65     48788
          13       0.74      0.71      0.72     20266
          14       0.74      0.58      0.65      5654

    accuracy                           0.61    312595
   macro avg       0.62      0.56      0.59    312595
weighted avg       0.61   

In [52]:
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

model = OneVsRestClassifier(MultinomialNB()).fit(X_train, Y_train)
pred = model.predict(X_test)

In [53]:
from sklearn.metrics import f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
print(classification_report(Y_test, pred))

              precision    recall  f1-score   support

           0       1.00      0.01      0.03      1798
           1       0.76      0.25      0.37     21317
           2       0.46      0.72      0.56     50768
           3       0.96      0.43      0.60      7065
           4       0.73      0.25      0.38     11649
           5       0.66      0.15      0.24     19054
           6       0.84      0.28      0.41     12020
           7       0.77      0.45      0.57     11296
           8       0.86      0.12      0.22      9236
           9       0.38      0.82      0.52     48035
          10       0.62      0.51      0.56     22057
          11       0.74      0.33      0.46     23592
          12       0.53      0.62      0.57     48788
          13       0.83      0.50      0.63     20266
          14       0.95      0.34      0.51      5654

    accuracy                           0.52    312595
   macro avg       0.74      0.39      0.44    312595
weighted avg       0.62   